In [1]:
import pandas as pd
import numpy as np
import torch

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/FintechDS

Mounted at /content/drive
/content/drive/My Drive/FintechDS


In [3]:
df = pd.read_csv("data/transformed/df.csv",index_col=False)
reviews_scraped = pd.read_csv("data/transformed/reviews_scraped.csv",index_col=False)
products_scraped = pd.read_csv("data/transformed/products_scraped.csv",index_col=False)

In [4]:
reviews_scraped.dropna(inplace= True)
products_scraped.dropna(inplace= True)

In [5]:
df.review_text = df.review_text.astype(str)
df.review_title = df.review_title.astype(str)
reviews_scraped.review_text = reviews_scraped.review_text.astype(str)

In [6]:
df.head()

,Unnamed: 0,rating,verified,PRODUCT_CATEGORY,product_id,review_title,review_text,cat_0,cat_1,cat_2,...,text_sentiment,text_subjectivity,rating_count,rating_avg,rating1,rating2,rating3,rating4,rating5,product_title
0,0,4.0,0.0,PC,B00008NG7N,useful,"When least you think so, this product will sav...",0.0,0.0,0.0,...,-0.300000,0.400000,233.0,4.6,0.03,0.01,0.07,0.07,0.81,"Targus PAUK10U Ultra Mini USB Keypad, Black"
1,1,4.0,1.0,Wireless,B00LH0Y3NM,New era for batteries,Lithium batteries are something new introduced...,0.0,0.0,0.0,...,0.219226,0.385320,214.0,3.8,0.17,0.06,0.14,0.11,0.53,Note 3 Battery : Stalion Strength Replacement ...
2,2,4.0,1.0,Wireless,B00LH0Y3NM,OK,"Seems to perform just ok, it's a battery. Came...",0.0,0.0,0.0,...,0.477778,0.611111,214.0,3.8,0.17,0.06,0.14,0.11,0.53,Note 3 Battery : Stalion Strength Replacement ...
3,3,3.0,0.0,Baby,B000I5UZ1Q,doesn't swing very well.,I purchased this swing for my baby. She is 6 m...,0.0,0.0,1.0,...,0.347143,0.600000,923.0,4.4,0.06,0.03,0.08,0.07,0.76,"Fisher-Price Papasan Cradle Swing, Starlight"
4,4,4.0,0.0,Office Products,B003822IRA,Great computing!,I was looking for an inexpensive desk calcolat...,0.0,0.0,0.0,...,-0.088542,0.441667,11026.0,4.8,0.01,0.01,0.03,0.10,0.85,Casio MS-80B Standard Function Desktop Calculator


In [7]:
reviews_scraped.head()

,Unnamed: 0,product_id,review_title,rating_count,rating_avg,rating1,rating2,rating3,rating4,rating5,verified,review_text,rating,text_sentiment,text_subjectivity
0,0,0006486118,"A Dance With Dragons (A Song of Ice and Fire, ...",17539.0,4.5,0.03,0.03,0.07,0.14,0.73,1.0,"To preface this review, I wanted to inform you...",5,0.124998,0.521898
1,1,0006486118,"A Dance With Dragons (A Song of Ice and Fire, ...",17539.0,4.5,0.03,0.03,0.07,0.14,0.73,1.0,Fans have been looking forward to this one for...,4,0.107453,0.520954
2,2,0028703200,Research in Music Education: An Introduction t...,7.0,4.8,0.00,0.00,0.00,0.23,0.77,1.0,Excellent book for me- I'm a bit rusty on rese...,5,0.460000,0.430000
3,3,0028703200,Research in Music Education: An Introduction t...,7.0,4.8,0.00,0.00,0.00,0.23,0.77,0.0,It is a fantastic book because it deals specif...,5,0.260000,0.312500
4,4,0060159839,James Mason: Odd Man Out,279.0,3.9,0.03,0.09,0.18,0.33,0.36,1.0,not the authors best work,4,1.000000,0.300000


In [26]:
products_scraped.head()

,Unnamed: 0,product_id,product_title,rating_count,rating_avg,rating1,rating2,rating3,rating4,rating5
0,0,0006486118,"A Dance With Dragons (A Song of Ice and Fire, ...",17539.0,4.5,0.03,0.03,0.07,0.14,0.73
1,1,0028703200,Research in Music Education: An Introduction t...,7.0,4.8,0.00,0.00,0.00,0.23,0.77
2,2,0060159839,James Mason: Odd Man Out,279.0,3.9,0.03,0.09,0.18,0.33,0.36
3,3,0060657677,A Hunger for Healing: The Twelve Steps as a Cl...,157.0,4.6,0.03,0.02,0.07,0.12,0.76
4,4,006074068X,Death at La Fenice: A Commissario Guido Brunet...,5284.0,4.4,0.01,0.02,0.10,0.31,0.56


In [30]:
reviews_scraped = pd.merge(products_scraped[['product_id','product_title']], reviews_scraped, on = 'product_id')

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data split

In [9]:
from sklearn.model_selection import train_test_split

x = df.loc[:, df.columns != 'label']
y = df.label

x_train, x_val, y_train, y_val = train_test_split( x, y, test_size=0.2,shuffle=True, random_state=1)
x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size=0.5,shuffle=True, random_state=1)

In [31]:
reviews_scraped_train = pd.merge(x_train.product_id, reviews_scraped, on = 'product_id')
reviews_scraped_val = pd.merge(x_val.product_id, reviews_scraped, on = 'product_id')
reviews_scraped_test = pd.merge(x_test.product_id, reviews_scraped, on = 'product_id')


# Model definition

In [11]:
class OurModelParameters():
    def __init__(self, bert):
      self.bert = bert
      pass

class OurModel(torch.nn.Module):

    def __init__(self, parameters: OurModelParameters):
      self.dropout = nn.Dropout(0.1)
      self.relu =  nn.ReLU()
      self.fc1 = nn.Linear(768,512)
      self.fc2 = nn.Linear(512,2)
      self.softmax = nn.LogSoftmax(dim=1)


      pass

    def forward(self, row_in_fake_real_products_df):
        """"
            row_in_fake_real_products_df is a df with the following columns:
                - product_title
                - review_title
                - rating1, rating2, rating3, rating4, rating5 (the percentages of ratings from each brackets, as values between 0.0 and 1.0)
                - 
        """
        pass
        

In [12]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 4.1 MB/s 
     |████████████████████████████████| 5.8 MB 59.4 MB/s 
     |████████████████████████████████| 1.3 MB 52.7 MB/s 
     |████████████████████████████████| 182 kB 64.9 MB/s 
     |████████████████████████████████| 7.6 MB 42.9 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=f8b6a3b974eb11852102ceadc9ea51f01aba94bcface1da24357b10d6457a3fb
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


# Text Embeddings

In [13]:
import transformers
from transformers import BertTokenizerFast,AutoTokenizer, BertTokenizer, AutoModel
#importing automodel didn't work with VSCode

bert = AutoModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [14]:
from tqdm.notebook import tqdm_notebook
def prepareDataForBert(data):
      tokenized = []

      pbar = tqdm_notebook(total = len(data))
      pbar.set_description(f'Tokenized: ')
      for i in data:
        tokenized.append(tokenizer.encode(i, add_special_tokens = True, truncation=True))
        pbar.update(1)
      pbar.close()
    
      max_len = 0
      for i in tokenized:
        if len(i) > max_len:
          max_len = len(i)

      padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized])
      attention_mask = np.where(padded != 0, 1, 0)
      
      return padded, attention_mask

In [15]:
idreviews_scraped = reviews_scraped['product_id'].unique()
data = []
for id in idreviews_scraped:
    reviews = reviews_scraped.loc[reviews_scraped['product_id'] == id].values
    data.append(reviews)

data = np.asarray(data)

<ipython-input-15-d6e1e6e7477a>:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data = np.asarray(data)


In [16]:
new_product_reviews_scraped = pd.DataFrame({'id': idreviews_scraped, 'product_reviews_scraped':data})
new_product_reviews_scraped.head(2)

,id,product_reviews_scraped
0,0006486118,"[[0, 0006486118, A Dance With Dragons (A Song ..."
1,0028703200,"[[2, 0028703200, Research in Music Education: ..."


# Scrapped review embeddings

In [25]:
#reviews_scrapped_ids, reviews_scrapped_mask
reviews_scrapped_ids_train, reviews_scrapped_mask_train = prepareDataForBert(reviews_scraped_train.review_text)
reviews_scrapped_ids_val,   reviews_scrapped_mask_val = prepareDataForBert(reviews_scraped_val.review_text)
reviews_scrapped_ids_test,  reviews_scrapped_mask_test = prepareDataForBert(reviews_scraped_test.review_text)


#reviews_scrapped_title_ids, reviews_scrapped_title_mask
reviews_scrapped_title_ids_train, reviews_scrapped_title_mask_train = prepareDataForBert(reviews_scraped_train.review_title)
reviews_scrapped_title_ids_val,   reviews_scrapped_title_mask_val = prepareDataForBert(reviews_scraped_val.review_title)
reviews_scrapped_title_ids_test,  reviews_scrapped_title_mask_test = prepareDataForBert(reviews_scraped_test.review_title)




  0%|          | 0/87083 [00:00<?, ?it/s]

  0%|          | 0/10908 [00:00<?, ?it/s]

  0%|          | 0/10652 [00:00<?, ?it/s]

  0%|          | 0/87083 [00:00<?, ?it/s]

  0%|          | 0/10908 [00:00<?, ?it/s]

  0%|          | 0/10652 [00:00<?, ?it/s]

In [34]:
#Product Title[str, ...] (BERT) => product_title_ids, product_title_mask
df_Product_Title_scrapped_ids_train, df_Product_Title_scrapped_mask_train = prepareDataForBert(reviews_scraped_train.product_title)
df_Product_Title_scrapped_ids_val,   df_Product_Title_scrapped_mask_val= prepareDataForBert(reviews_scraped_val.product_title)
df_Product_Title_scrapped_ids_test,  df_Product_Title_scrapped_mask_test = prepareDataForBert(reviews_scraped_test.product_title)


  0%|          | 0/87083 [00:00<?, ?it/s]

  0%|          | 0/10908 [00:00<?, ?it/s]

  0%|          | 0/10652 [00:00<?, ?it/s]

In [18]:
df_Review_Text_ids_train, df_Review_Text_train = prepareDataForBert(x_train.review_text)
df_Review_Text_ids_val,   df_Review_Text_val= prepareDataForBert(x_val.review_text)
df_Review_Text_ids_test,  df_Review_Text_test = prepareDataForBert(x_test.review_text) 

  0%|          | 0/14485 [00:00<?, ?it/s]

  0%|          | 0/1811 [00:00<?, ?it/s]

  0%|          | 0/1811 [00:00<?, ?it/s]

# Normal Review embeddings (not scrapped)


In [19]:
df_Review_Text_ids_train, df_Review_Text_train = prepareDataForBert(x_train.review_text)
df_Review_Text_ids_val,   df_Review_Text_val= prepareDataForBert(x_val.review_text)
df_Review_Text_ids_test,  df_Review_Text_test = prepareDataForBert(x_test.review_text) 

  0%|          | 0/14485 [00:00<?, ?it/s]

  0%|          | 0/1811 [00:00<?, ?it/s]

  0%|          | 0/1811 [00:00<?, ?it/s]

## Product Title embeddings (not scrapped)

In [20]:
#Product Title[str, ...] (BERT) => product_title_ids, product_title_mask
df_Product_Title_ids_train, df_Product_Title_mask_train = prepareDataForBert(x_train.product_title)
df_Product_Title_ids_val,   df_Product_Title_mask_val= prepareDataForBert(x_val.product_title)
df_Product_Title_ids_test,  df_Product_Title_mask_test = prepareDataForBert(x_test.product_title)


  0%|          | 0/14485 [00:00<?, ?it/s]

  0%|          | 0/1811 [00:00<?, ?it/s]

  0%|          | 0/1811 [00:00<?, ?it/s]

# Create Torch Dataset

In [24]:
from torch.utils.data import Dataset, DataLoader

class DatasetClass(Dataset):
    ''' 
    @ Name Column  =>  @ Name Variable

    Review Embedding Network
        Review_Title (BERT)=>  df_Review_Title_ids, df_Review_Title_mask
        Review_Text (BERT)=>   df_Review_Text_ids, df_Review_Text_mask
        Rating, Verified Purchase, SA Valence, Subjectivity => Rating_Purchase_Subjectivity

    Black Box 
        Review(Scraped) (BERT)=> reviews_scrapped_ids, reviews_scrapped_mask
        Review_Title(Scraped) (BERT)=> reviews_scrapped_title_ids, reviews_scrapped_title_mask

    Product Title[str, ...] (BERT) => product_title_ids, product_title_mask

    Ratings, Ratio, RatingAvg, Rating Count => ratings

    Category (One hot encoding) => category 


    ----------------------------------------------------------------------------------------------------------------------------
        
    n_samples => Obtained from df dataframe col: PRODUCT_CATEGORY
    '''
    def __init__(
        self, 

        # Review Embedding Network
        df_Review_Title_ids, 
        df_Review_Title_mask,

        df_Review_Text_ids,
        df_Review_Text_mask,

        Rating_Purchase_Subjectivity,

        reviews_scrapped_ids,
        reviews_scrapped_mask,

        reviews_scrapped_title_ids, 
        reviews_scrapped_title_mask,

        product_title_ids,
        product_title_mask,

        ratings,

        category,

        ):

        self.df_Review_Title_ids =  torch.tensor(df_Review_Title_ids)
        self.df_Review_Title_mask =  torch.tensor(df_Review_Title_mask)

        self.df_Review_Text_ids =  torch.tensor(df_Review_Text_ids)
        self.df_Review_Text_mask =  torch.tensor(df_Review_Text_mask)

        self.reviews_scrapped_title_ids =  torch.tensor(reviews_scrapped_title_ids)
        self.reviews_scrapped_title_mask= torch.tensor(reviews_scrapped_title_mask)

        self.Rating_Purchase_Subjectivity = torch.from_numpy(np.asarray(Rating_Purchase_Subjectivity, dtype=np.float32))

        self.reviews_scrapped_ids =  torch.tensor(reviews_scrapped_ids)
        self.reviews_scrapped_mask =  torch.tensor(reviews_scrapped_mask)

        self.product_title_ids =  torch.tensor(product_title_ids)
        self.product_title_mask =  torch.tensor(product_title_mask)

        self.ratings = torch.from_numpy(np.asarray(ratings, dtype=np.float32))

        self.category = torch.from_numpy(np.asarray(category, dtype=np.int8))


        self.n_samples = len(category)

    def __getitem__(self, index):
        return self.df_Review_Title_ids[index], self.df_Review_Title_mask[index], \
            self.df_Review_Text_ids[index], self.df_Review_Text_mask[index], \
            self.Rating_Purchase_Subjectivity[index], \
            self.reviews_scrapped_ids[index], self.reviews_scrapped_mask[index], \
            self.reviews_scrapped__title_ids[index], self.reviews_scrapped_title_mask[index], \
            self.product_title_ids[index], self.product_title_mask[index], \
            self.ratings[index], \
            self.category[index]

    def __len__(self):
        return self.n_samples 

# Create dataset class objects

In [35]:


train = DatasetClass(
            df_Review_Title_ids_train, 
            df_Review_Title_mask_train,    
            df_Review_Text_ids_train,
            df_Review_Text_mask_train,    
            Rating_Purchase_Subjectivity, 
            reviews_scrapped_ids_train,
            reviews_scrapped_mask_train,
            reviews_scrapped_title_ids_train, 
            reviews_scrapped_title_mask_train,               
            product_title_ids_train,
            product_title_mask_train,    
            ratings_train,    # categorical columns
            category_train,
            )

validate = DatasetClass(
            df_Review_Title_ids_val, 
            df_Review_Title_mask_val,    
            df_Review_Text_ids_val,
            df_Review_Text_mask_val,    
            Rating_Purchase_Subjectivity,    
            reviews_scraped_ids_val,
            reviews_scraped_mask_val,    
            product_title_ids_val,
            product_title_mask_val,    
            ratings_val,    # categorical columns
            category_val,
            )

test = DatasetClass(
            df_Review_Title_ids_test, 
            df_Review_Title_mask_test,    
            df_Review_Text_ids_test,
            df_Review_Text_mask_test,    
            Rating_Purchase_Subjectivity,    
            reviews_scraped_ids_test,
            reviews_scraped_mask_test,    
            product_title_ids_test,
            product_title_mask_test,    
            ratings_test,    # categorical columns
            category_test,
            )

NameError: ignored